In [1]:
!pip install kaggle

In [0]:
!mkdir .kaggle

In [0]:
import json
token = {"username":"sushmita1794","key":"863998e4922d86a43de1845b384363d1"}

with open('/content/.kaggle/kaggle.json', 'w') as file: json.dump(token, file)


In [0]:
!chmod 600 /content/.kaggle/kaggle.json

In [0]:
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json

In [9]:
!kaggle config set -n path -v{/content}

- path is now set to: {/content}


In [10]:
!kaggle datasets download -d usdot/flight-delays

 99% 189M/191M [00:04<00:00, 33.3MB/s]
100% 191M/191M [00:04<00:00, 40.1MB/s]


In [11]:
!unzip /content/sample_data/flight-delays.zip

Archive:  /content/sample_data/flight-delays.zip
  inflating: airlines.csv            
  inflating: airports.csv            
  inflating: flights.csv             


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time


In [13]:
#load the csv file and use the first 100000 rows of data
df_use = pd.read_csv('flights.csv')
df = df_use.sample(n=100000).copy()
del df_use

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [14]:
df.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
4108657,2015,9,13,7,WN,1986,N610WN,SLC,OAK,1210,1219.0,9.0,14.0,1233.0,110.0,119.0,100.0,588,1313.0,5.0,1300,1318.0,18.0,0,0,NaN,16.0,0.0,0.0,2.0,0.0
3803873,2015,8,24,1,EV,4666,N16918,IAH,AMA,1738,1732.0,-6.0,14.0,1746.0,106.0,97.0,79.0,517,1905.0,4.0,1924,1909.0,-15.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
36407,2015,1,3,6,NK,511,N533NK,DTW,LAS,1117,1122.0,5.0,33.0,1155.0,269.0,283.0,242.0,1749,1257.0,8.0,1246,1305.0,19.0,0,0,NaN,19.0,0.0,0.0,0.0,0.0
4301851,2015,9,25,5,AA,2142,N768US,BOS,DCA,1500,1457.0,-3.0,13.0,1510.0,94.0,92.0,76.0,399,1626.0,3.0,1634,1629.0,-5.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2153423,2015,5,17,7,AA,2324,N4XLAA,DFW,RIC,1355,1350.0,-5.0,16.0,1406.0,175.0,169.0,150.0,1158,1736.0,3.0,1750,1739.0,-11.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
df.groupby('DEPARTURE_DELAY').size()

DEPARTURE_DELAY
-40.0      1
-39.0      1
-35.0      1
-32.0      1
-31.0      1
          ..
 1205.0    1
 1341.0    1
 1362.0    1
 1375.0    1
 1625.0    1
Length: 475, dtype: int64

In [0]:
df = df.loc[~df.CANCELLED.isin([1])]

In [0]:
df['OUTPUT_LABEL'] = (df.DEPARTURE_DELAY >= 0).astype('int')

In [18]:
df_airports = pd.read_csv('airports.csv')
df_airports.head()

,IATA_CODE,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE
0,ABE,Lehigh Valley International Airport,Allentown,PA,USA,40.65236,-75.44040
1,ABI,Abilene Regional Airport,Abilene,TX,USA,32.41132,-99.68190
2,ABQ,Albuquerque International Sunport,Albuquerque,NM,USA,35.04022,-106.60919
3,ABR,Aberdeen Regional Airport,Aberdeen,SD,USA,45.44906,-98.42183
4,ABY,Southwest Georgia Regional Airport,Albany,GA,USA,31.53552,-84.19447


In [0]:
df.loc[~df.ORIGIN_AIRPORT.isin(df_airports.IATA_CODE.values), 'ORIGIN_AIRPORT'] = 'OTHER'
df.loc[~df.DESTINATION_AIRPORT.isin(df_airports.IATA_CODE.values), 'DESTINATION_AIRPORT'] = 'OTHER'

In [20]:
cols_num = ['SCHEDULED_DEPARTURE', 'SCHEDULED_TIME', 'DISTANCE', 'SCHEDULED_ARRIVAL', 'DAY_OF_WEEK']
cols_num
df[cols_num].isnull().sum()

SCHEDULED_DEPARTURE    0
SCHEDULED_TIME         0
DISTANCE               0
SCHEDULED_ARRIVAL      0
DAY_OF_WEEK            0
dtype: int64

In [21]:
cols_cat = ['AIRLINE', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT']
cols_cat
df[cols_cat].isnull().sum()

AIRLINE                0
ORIGIN_AIRPORT         0
DESTINATION_AIRPORT    0
dtype: int64

In [22]:
df[cols_cat].nunique()

AIRLINE                 14
ORIGIN_AIRPORT         321
DESTINATION_AIRPORT    320
dtype: int64

In [0]:
df_cat = pd.get_dummies(df[cols_cat], drop_first=True)

In [24]:
df_cat.head()

,AIRLINE_AS,AIRLINE_B6,AIRLINE_DL,AIRLINE_EV,AIRLINE_F9,AIRLINE_HA,AIRLINE_MQ,AIRLINE_NK,AIRLINE_OO,AIRLINE_UA,AIRLINE_US,AIRLINE_VX,AIRLINE_WN,ORIGIN_AIRPORT_ABI,ORIGIN_AIRPORT_ABQ,ORIGIN_AIRPORT_ABR,ORIGIN_AIRPORT_ABY,ORIGIN_AIRPORT_ACK,ORIGIN_AIRPORT_ACT,ORIGIN_AIRPORT_ACV,ORIGIN_AIRPORT_ACY,ORIGIN_AIRPORT_ADK,ORIGIN_AIRPORT_ADQ,ORIGIN_AIRPORT_AEX,ORIGIN_AIRPORT_AGS,ORIGIN_AIRPORT_ALB,ORIGIN_AIRPORT_ALO,ORIGIN_AIRPORT_AMA,ORIGIN_AIRPORT_ANC,ORIGIN_AIRPORT_APN,ORIGIN_AIRPORT_ASE,ORIGIN_AIRPORT_ATL,ORIGIN_AIRPORT_ATW,ORIGIN_AIRPORT_AUS,ORIGIN_AIRPORT_AVL,ORIGIN_AIRPORT_AVP,ORIGIN_AIRPORT_AZO,ORIGIN_AIRPORT_BDL,ORIGIN_AIRPORT_BET,ORIGIN_AIRPORT_BFL,...,DESTINATION_AIRPORT_SHV,DESTINATION_AIRPORT_SIT,DESTINATION_AIRPORT_SJC,DESTINATION_AIRPORT_SJT,DESTINATION_AIRPORT_SJU,DESTINATION_AIRPORT_SLC,DESTINATION_AIRPORT_SMF,DESTINATION_AIRPORT_SMX,DESTINATION_AIRPORT_SNA,DESTINATION_AIRPORT_SPI,DESTINATION_AIRPORT_SPS,DESTINATION_AIRPORT_SRQ,DESTINATION_AIRPORT_STL,DESTINATION_AIRPORT_STT,DESTINATION_AIRPORT_STX,DESTINATION_AIRPORT_SUN,DESTINATION_AIRPORT_SUX,DESTINATION_AIRPORT_SWF,DESTINATION_AIRPORT_SYR,DESTINATION_AIRPORT_TLH,DESTINATION_AIRPORT_TOL,DESTINATION_AIRPORT_TPA,DESTINATION_AIRPORT_TRI,DESTINATION_AIRPORT_TTN,DESTINATION_AIRPORT_TUL,DESTINATION_AIRPORT_TUS,DESTINATION_AIRPORT_TVC,DESTINATION_AIRPORT_TWF,DESTINATION_AIRPORT_TXK,DESTINATION_AIRPORT_TYR,DESTINATION_AIRPORT_TYS,DESTINATION_AIRPORT_UST,DESTINATION_AIRPORT_VEL,DESTINATION_AIRPORT_VLD,DESTINATION_AIRPORT_VPS,DESTINATION_AIRPORT_WRG,DESTINATION_AIRPORT_WYS,DESTINATION_AIRPORT_XNA,DESTINATION_AIRPORT_YAK,DESTINATION_AIRPORT_YUM
4108657,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3803873,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
36407,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4301851,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2153423,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
df = pd.concat([df, df_cat], axis = 1)

In [26]:
cols_all_cat = list(df_cat.columns)
print('number of categorical: ', len(cols_all_cat))

number of categorical:  652


In [27]:
cols_input = cols_num + cols_all_cat
df_data = df[cols_input + ['OUTPUT_LABEL']]
df_data.head()

,SCHEDULED_DEPARTURE,SCHEDULED_TIME,DISTANCE,SCHEDULED_ARRIVAL,DAY_OF_WEEK,AIRLINE_AS,AIRLINE_B6,AIRLINE_DL,AIRLINE_EV,AIRLINE_F9,AIRLINE_HA,AIRLINE_MQ,AIRLINE_NK,AIRLINE_OO,AIRLINE_UA,AIRLINE_US,AIRLINE_VX,AIRLINE_WN,ORIGIN_AIRPORT_ABI,ORIGIN_AIRPORT_ABQ,ORIGIN_AIRPORT_ABR,ORIGIN_AIRPORT_ABY,ORIGIN_AIRPORT_ACK,ORIGIN_AIRPORT_ACT,ORIGIN_AIRPORT_ACV,ORIGIN_AIRPORT_ACY,ORIGIN_AIRPORT_ADK,ORIGIN_AIRPORT_ADQ,ORIGIN_AIRPORT_AEX,ORIGIN_AIRPORT_AGS,ORIGIN_AIRPORT_ALB,ORIGIN_AIRPORT_ALO,ORIGIN_AIRPORT_AMA,ORIGIN_AIRPORT_ANC,ORIGIN_AIRPORT_APN,ORIGIN_AIRPORT_ASE,ORIGIN_AIRPORT_ATL,ORIGIN_AIRPORT_ATW,ORIGIN_AIRPORT_AUS,ORIGIN_AIRPORT_AVL,...,DESTINATION_AIRPORT_SIT,DESTINATION_AIRPORT_SJC,DESTINATION_AIRPORT_SJT,DESTINATION_AIRPORT_SJU,DESTINATION_AIRPORT_SLC,DESTINATION_AIRPORT_SMF,DESTINATION_AIRPORT_SMX,DESTINATION_AIRPORT_SNA,DESTINATION_AIRPORT_SPI,DESTINATION_AIRPORT_SPS,DESTINATION_AIRPORT_SRQ,DESTINATION_AIRPORT_STL,DESTINATION_AIRPORT_STT,DESTINATION_AIRPORT_STX,DESTINATION_AIRPORT_SUN,DESTINATION_AIRPORT_SUX,DESTINATION_AIRPORT_SWF,DESTINATION_AIRPORT_SYR,DESTINATION_AIRPORT_TLH,DESTINATION_AIRPORT_TOL,DESTINATION_AIRPORT_TPA,DESTINATION_AIRPORT_TRI,DESTINATION_AIRPORT_TTN,DESTINATION_AIRPORT_TUL,DESTINATION_AIRPORT_TUS,DESTINATION_AIRPORT_TVC,DESTINATION_AIRPORT_TWF,DESTINATION_AIRPORT_TXK,DESTINATION_AIRPORT_TYR,DESTINATION_AIRPORT_TYS,DESTINATION_AIRPORT_UST,DESTINATION_AIRPORT_VEL,DESTINATION_AIRPORT_VLD,DESTINATION_AIRPORT_VPS,DESTINATION_AIRPORT_WRG,DESTINATION_AIRPORT_WYS,DESTINATION_AIRPORT_XNA,DESTINATION_AIRPORT_YAK,DESTINATION_AIRPORT_YUM,OUTPUT_LABEL
4108657,1210,110.0,588,1300,7,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3803873,1738,106.0,517,1924,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
36407,1117,269.0,1749,1246,6,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4301851,1500,94.0,399,1634,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2153423,1355,175.0,1158,1750,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
df_data = df_data.sample(n = len(df_data))
df_data = df_data.reset_index(drop = True)

In [29]:
df_valid_test = df_data.sample(frac=0.30)
print('Split size: %.3f' %(len(df_valid_test)/len(df_data)))

Split size: 0.300


In [0]:
df_test = df_valid_test.sample(frac = 0.5)
df_valid = df_valid_test.drop(df_test.index)

In [0]:
df_train_all = df_data.drop(df_valid_test.index)

In [32]:
print('Test n = %d' %(len(df_test)))
print('Valid n = %d' %(len(df_valid)))
print('Train n = %d' %(len(df_train_all)))
df_test

Test n = 14772
Valid n = 14772
Train n = 68936


,SCHEDULED_DEPARTURE,SCHEDULED_TIME,DISTANCE,SCHEDULED_ARRIVAL,DAY_OF_WEEK,AIRLINE_AS,AIRLINE_B6,AIRLINE_DL,AIRLINE_EV,AIRLINE_F9,AIRLINE_HA,AIRLINE_MQ,AIRLINE_NK,AIRLINE_OO,AIRLINE_UA,AIRLINE_US,AIRLINE_VX,AIRLINE_WN,ORIGIN_AIRPORT_ABI,ORIGIN_AIRPORT_ABQ,ORIGIN_AIRPORT_ABR,ORIGIN_AIRPORT_ABY,ORIGIN_AIRPORT_ACK,ORIGIN_AIRPORT_ACT,ORIGIN_AIRPORT_ACV,ORIGIN_AIRPORT_ACY,ORIGIN_AIRPORT_ADK,ORIGIN_AIRPORT_ADQ,ORIGIN_AIRPORT_AEX,ORIGIN_AIRPORT_AGS,ORIGIN_AIRPORT_ALB,ORIGIN_AIRPORT_ALO,ORIGIN_AIRPORT_AMA,ORIGIN_AIRPORT_ANC,ORIGIN_AIRPORT_APN,ORIGIN_AIRPORT_ASE,ORIGIN_AIRPORT_ATL,ORIGIN_AIRPORT_ATW,ORIGIN_AIRPORT_AUS,ORIGIN_AIRPORT_AVL,...,DESTINATION_AIRPORT_SIT,DESTINATION_AIRPORT_SJC,DESTINATION_AIRPORT_SJT,DESTINATION_AIRPORT_SJU,DESTINATION_AIRPORT_SLC,DESTINATION_AIRPORT_SMF,DESTINATION_AIRPORT_SMX,DESTINATION_AIRPORT_SNA,DESTINATION_AIRPORT_SPI,DESTINATION_AIRPORT_SPS,DESTINATION_AIRPORT_SRQ,DESTINATION_AIRPORT_STL,DESTINATION_AIRPORT_STT,DESTINATION_AIRPORT_STX,DESTINATION_AIRPORT_SUN,DESTINATION_AIRPORT_SUX,DESTINATION_AIRPORT_SWF,DESTINATION_AIRPORT_SYR,DESTINATION_AIRPORT_TLH,DESTINATION_AIRPORT_TOL,DESTINATION_AIRPORT_TPA,DESTINATION_AIRPORT_TRI,DESTINATION_AIRPORT_TTN,DESTINATION_AIRPORT_TUL,DESTINATION_AIRPORT_TUS,DESTINATION_AIRPORT_TVC,DESTINATION_AIRPORT_TWF,DESTINATION_AIRPORT_TXK,DESTINATION_AIRPORT_TYR,DESTINATION_AIRPORT_TYS,DESTINATION_AIRPORT_UST,DESTINATION_AIRPORT_VEL,DESTINATION_AIRPORT_VLD,DESTINATION_AIRPORT_VPS,DESTINATION_AIRPORT_WRG,DESTINATION_AIRPORT_WYS,DESTINATION_AIRPORT_XNA,DESTINATION_AIRPORT_YAK,DESTINATION_AIRPORT_YUM,OUTPUT_LABEL
7423,1251,146.0,925,1617,4,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
78598,1126,153.0,946,1359,5,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
10911,1355,84.0,391,1519,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
81906,1905,180.0,1096,2205,3,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
56895,1220,158.0,1139,1558,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36143,1820,108.0,547,2008,2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
49668,1405,95.0,488,1440,2,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
69235,1530,115.0,672,1725,2,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
13950,1200,151.0,852,1431,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [33]:
rows_pos = df_train_all.OUTPUT_LABEL == 1
df_train_pos = df_train_all.loc[rows_pos]
df_train_neg = df_train_all.loc[~rows_pos]

df_train = pd.concat([df_train_pos, df_train_neg.sample(n=len(df_train_pos))], axis=0)

df_train = df_train.sample(n=len(df_train)).reset_index(drop=True)

print('Train balanced prevalence(n= %d)'%(len(df_train)))

Train balanced prevalence(n= 59124)


In [34]:
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import accuracy_score
import lightgbm as lgb
y = df_data.OUTPUT_LABEL
X = df_data.drop(['OUTPUT_LABEL'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

dtrain=xgb.DMatrix(X_train,label=y_train)
dtest=xgb.DMatrix(X_test)
parameters={'max_depth':7, 'silent':1,'objective':'binary:logistic','eval_metric':'auc','learning_rate':.05}

/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


In [0]:
class Vertex:
    def __init__(self, node):
        self.id = node
        self.adjacent = {}

    def __str__(self):
        return str(self.id) + ' adjacent: ' + str([x.id for x in self.adjacent])

    def add_neighbor(self, neighbor, weight=0):
        self.adjacent[neighbor] = weight

    def get_connections(self):
        return self.adjacent.keys()  

    def get_id(self):
        return self.id

    def get_weight(self, neighbor):
        return self.adjacent[neighbor]

class Graph:
    def __init__(self):
        self.vert_dict = {}
        self.num_vertices = 0

    def __iter__(self):
        return iter(self.vert_dict.values())

    def add_vertex(self, node):
        self.num_vertices = self.num_vertices + 1
        new_vertex = Vertex(node)
        self.vert_dict[node] = new_vertex
        return new_vertex

    def get_vertex(self, n):
        if n in self.vert_dict:
            return self.vert_dict[n]
        else:
            return None

    def add_edge(self, frm, to, cost = 0):
        if frm not in self.vert_dict:
            self.add_vertex(frm)
        if to not in self.vert_dict:
            self.add_vertex(to)

        self.vert_dict[frm].add_neighbor(self.vert_dict[to], cost)
        self.vert_dict[to].add_neighbor(self.vert_dict[frm], cost)

    def get_vertices(self):
        return self.vert_dict.keys()

if __name__ == '__main__':

    g = Graph()
    

In [0]:
new_features = X.to_numpy()
num_cols = np.size(new_features, 1)
for i in range(num_cols):
    g.add_vertex(i)
indices=[]
for i in range(num_cols):
  indices.append(np.nonzero(new_features[:,i]))

indices_num = np.size(indices,0)

for i in range(indices_num-1):
  first_feature = indices[i][0]
  for j in range(i+1, indices_num):
    second_feature = indices[j][0]
    count=0
    a=0
    b=0
    while a < first_feature.size and b < second_feature.size:
      if(first_feature[a]==second_feature[b]):
        count+=1
        a+=1
        b+=1
      elif(first_feature[a]>second_feature[b]):
        b+=1
      else:
        a+=1
    g.add_edge(i, j, count)

In [0]:
F=new_features
bundles = []
conflicts=[]
for i in range(1, np.size(F,1)):
  conflicts.append(np.count_nonzero(F[:,i]))

sorted_indices=np.argsort(conflicts)
i = 0

while i <= len(conflicts) - 154:
  bundle = []
  for k in range(0, 5):
    bundle.append(list(F[:,sorted_indices[i+k]]))
  bundles.append(bundle)
  del(bundle)
  i += 5


In [38]:
final_features=[]
for j in range(len(bundles)):
  F0=np.asarray(bundles[j][0])
  F1=np.asarray(bundles[j][1])
  F2=np.asarray(bundles[j][2])
  F3=np.asarray(bundles[j][3])
  F4=np.asarray(bundles[j][4])

  F1_ind=np.nonzero(F1)[0]
  (F1_ind[0])
  F2_ind=np.nonzero(F2)[0]
  F3_ind=np.nonzero(F3)[0]
  F4_ind=np.nonzero(F4)[0]
  for i in range(len(F1_ind)):
    if(F0[F1_ind[i]])==0:
      F0[F1_ind[i]]=F1[F1_ind[i]]+2

  for i in range(len(F2_ind)):
    if(F0[F2_ind[i]])==0:
      F0[F2_ind[i]]=F2[F2_ind[i]]+4

  for i in range(len(F3_ind)):
    if(F0[F3_ind[i]])==0:
      F0[F3_ind[i]]=F3[F3_ind[i]]+6

  for i in range(len(F4_ind)):
    if(F0[F4_ind[i]])==0:
      F0[F4_ind[i]]=F4[F4_ind[i]]+8  
  
  final_features.append(F0)

len(final_features)
type(final_features)
final_features

X_efb1=np.asarray(final_features)
X_efb2=X_efb1.transpose()
X_efb3=[]
for m in range(500, 654):
  X_efb3.append(F[:,sorted_indices[m]])


X_efb33=(np.asarray(X_efb3)).transpose()

X_efb33.shape
X_efb2.shape

X_efb=np.concatenate(( X_efb2,X_efb33), axis=1)
X_efb.shape

(98480, 255)

In [43]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score
from datetime import datetime 
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import confusion_matrix 

ada = AdaBoostClassifier(n_estimators=100, random_state=0)
X_train_efb, X_test_efb, y_train_efb, y_test_efb = train_test_split(X_efb, y, test_size=0.2, random_state=123)
start=datetime.now()
ada.fit(X_train_efb, y_train_efb)
y_predict_efb=ada.predict(X_test_efb)
stop=datetime.now()
execution_time_efb = stop-start
print(execution_time_efb)
print(accuracy_score(y_test_efb, y_predict_efb))
results = confusion_matrix(y_test_efb, y_predict_efb) 
print(results)


0:00:51.268390
0.6280463038180342
[[8864 2638]
 [4688 3506]]
